<a href="https://colab.research.google.com/github/19K41A04H5/NNDL/blob/main/ass3_nndl_4h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel(os.path.join(os.getcwd(), 'Assets', '/content/drive/MyDrive/Load Data in kW (3).xlsx'))
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [4]:
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-1]) #Since x is previous hour of y
    y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Previous Hour Load' : x, 'Present Hour Load' : y})
data.head()

,Previous Hour Load,Present Hour Load
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


In [6]:
maxX = np.max(data['Previous Hour Load'])
minX = np.min(data['Previous Hour Load'])
maxY = np.max(data['Present Hour Load'])
minY = np.min(data['Present Hour Load'])

data['Previous Hour Load'] = (data['Previous Hour Load'] - minX) / (maxX - minX)
data['Present Hour Load'] = (data['Present Hour Load'] - minY) / (maxY - minY)

In [7]:
x = data['Previous Hour Load']
y = data['Present Hour Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [8]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        
        tempM = m + v*vm
        tempC = c + v*vc
        
        
        gradM = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC) * trainX.iloc[i]
    
        
        gradC = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC)
        
       
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        
        m += vm
        c += vc


print(f'm = {m}\nc = {c}')

m = 0.7799247301408075
c = 0.11205495172826278


In [9]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

#training error caculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 470.78544515956565
MSE : 430838.4062024668
RMSE : 656.3828198562686
Testing Error :
MAE : 516.1635560852443
MSE : 542768.9860455303
RMSE : 736.7285701298208


In [10]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,7272.11376,6798.534635
1,6082.74936,6550.279165
2,6436.59696,6624.682862
3,5349.80160,6263.606101
4,8121.65976,7451.390597
...,...,...
1959,5671.84968,5749.102110
1960,5442.86196,5957.967388
1961,4870.93824,5495.011056
1962,8138.65068,7769.429925


In [11]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,6052.82040,5875.782913
1,5190.80400,5700.958542
2,5097.27600,5466.319435
3,6263.57016,6892.147129
4,4543.90200,4979.777618
...,...,...
214,6172.84800,6517.697155
215,5864.20560,5325.900695
216,4667.82660,4228.689326
217,7943.64480,7748.032784


In [13]:
x = float(input('Enter the load at previous hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at previous hour : 3425.8765
Predicted load at present hour : 4027.562601349591
